In [28]:
import openmeteo_requests
import requests
import requests_cache
import pandas as pd
from retry_requests import retry

pd.set_option("display.max_rows", 50)

In [6]:

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [12]:
url = "https://geocoding-api.open-meteo.com/v1/search"
params = {
	"name": "Wallberg",
	"count": 1,
	"language": "de",
    "format": "json"
}
geo_responses = requests.get(url, params=params).json()

In [14]:
latitude = geo_responses["results"][0]["latitude"]
longitude = geo_responses["results"][0]["longitude"]

In [17]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": latitude,
	"longitude": longitude,
	"hourly": ["temperature_2m", "precipitation", "rain", "showers", "snowfall", "snow_depth", "cloud_cover", "cloud_cover_low", "visibility", "wind_speed_10m", "soil_moisture_0_to_1cm"],
	"daily": ["sunrise", "sunset", "sunshine_duration"],
	"timezone": "Europe/Berlin",
	"past_days": 14,
	"forecast_days": 14,
	"models": ["best_match", "icon_d2"]
}
responses = openmeteo.weather_api(url, params=params)


In [22]:

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_showers = hourly.Variables(3).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(5).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(6).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(7).ValuesAsNumpy()
hourly_visibility = hourly.Variables(8).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_moisture_0_to_1cm = hourly.Variables(10).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["showers"] = hourly_showers
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["visibility"] = hourly_visibility
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["soil_moisture_0_to_1cm"] = hourly_soil_moisture_0_to_1cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe


Coordinates 47.63999938964844°N 11.779998779296875°E
Elevation 1708.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s


,date,temperature_2m,precipitation,rain,showers,snowfall,snow_depth,cloud_cover,cloud_cover_low,visibility,wind_speed_10m,soil_moisture_0_to_1cm
0,2024-11-02 23:00:00+00:00,0.530500,0.0,0.000,0.0,0.0000,0.00,93.0,69.0,1000.0,2.276840,0.283
1,2024-11-03 00:00:00+00:00,-0.069500,0.0,0.000,0.0,0.0000,0.00,100.0,100.0,120.0,1.800000,0.283
2,2024-11-03 01:00:00+00:00,-0.019500,0.0,0.000,0.0,0.0000,0.00,100.0,100.0,120.0,5.804825,0.283
3,2024-11-03 02:00:00+00:00,1.280500,0.0,0.000,0.0,0.0000,0.00,100.0,100.0,280.0,4.610250,0.284
4,2024-11-03 03:00:00+00:00,1.030500,0.0,0.000,0.0,0.0000,0.00,100.0,100.0,180.0,2.902413,0.287
5,2024-11-03 04:00:00+00:00,0.680500,0.0,0.000,0.0,0.0000,0.00,100.0,100.0,180.0,1.297998,0.287
6,2024-11-03 05:00:00+00:00,0.080500,0.0,0.000,0.0,0.0000,0.00,100.0,100.0,200.0,1.080000,0.287
7,2024-11-03 06:00:00+00:00,1.430500,0.0,0.000,0.0,0.0000,0.00,18.0,0.0,11620.0,1.835647,0.287
8,2024-11-03 07:00:00+00:00,0.880500,0.0,0.000,0.0,0.0000,0.00,100.0,100.0,120.0,1.484318,0.287
9,2024-11-03 08:00:00+00:00,1.630500,0.0,0.000,0.0,0.0000,0.00,78.0,35.0,2700.0,2.811690,0.287


In [ ]:

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_sunrise = daily.Variables(0).ValuesAsNumpy()
daily_sunset = daily.Variables(1).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["sunshine_duration"] = daily_sunshine_duration

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe

In [29]:
hourly_dataframe[hourly_dataframe['snow_depth'] > 0]

,date,temperature_2m,precipitation,rain,showers,snowfall,snow_depth,cloud_cover,cloud_cover_low,visibility,wind_speed_10m,soil_moisture_0_to_1cm
261,2024-11-13 20:00:00+00:00,-5.4695,0.8,0.0,0.0,0.56,0.01,100.0,90.0,3140.0,7.594208,0.289
262,2024-11-13 21:00:00+00:00,-5.6695,0.4,0.0,0.0,0.14,0.01,99.0,91.0,5240.0,6.193674,0.289
263,2024-11-13 22:00:00+00:00,-5.8695,0.0,0.0,0.0,0.00,0.01,95.0,91.0,2980.0,5.001280,0.289
264,2024-11-13 23:00:00+00:00,-6.0195,0.0,0.0,0.0,0.00,0.02,100.0,100.0,700.0,3.545589,0.289
364,2024-11-18 03:00:00+00:00,-2.7195,0.8,0.0,0.0,0.14,0.01,100.0,100.0,440.0,8.647496,0.300
...,...,...,...,...,...,...,...,...,...,...,...,...
537,2024-11-25 08:00:00+00:00,-2.0840,0.0,0.0,0.0,0.00,0.04,81.0,0.0,24140.0,10.308830,NaN
538,2024-11-25 09:00:00+00:00,-0.4840,0.0,0.0,0.0,0.00,0.04,73.0,0.0,24140.0,9.504273,NaN
539,2024-11-25 10:00:00+00:00,0.6660,0.0,0.0,0.0,0.00,0.03,62.0,0.0,23860.0,9.028754,NaN
540,2024-11-25 11:00:00+00:00,1.5160,0.0,0.0,0.0,0.00,0.02,51.0,0.0,23580.0,8.699793,NaN
